In [1]:
import os
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize,sent_tokenize,RegexpTokenizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from collections import Counter

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')

import os
java_path = "C:/Program Files/Java/jre1.8.0_201/bin/java.exe"
os.environ['JAVAHOME'] = java_path

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\genec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\genec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\genec\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [284]:
articles = []
sentences = []
newSen= []

for file in os.listdir('data/allArticles/'):
    articles.append(open(os.path.join('data/allArticles',file),'rt',errors='ignore').read())

for i in range(len(articles)):
    sentences.append(sent_tokenize(articles[i]))
    
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        newSen.append(sentences[i][j])

#### Question analysis

In [1132]:
# input question
question = 'What percentage of drop or increase in GDP is associated with the fiscal cliff?'

In [1133]:
person = ['Who','Whom','Whose']
location = ['Where']
time = ['When','What time','What date']
nouns = ['What']
quantity = ['How few','How great','How many','How little','How much','What percentage']
organization = ['Which']
types = [nouns,person,location,time,quantity,organization]

In [1134]:
for item in types:
    if any(word in question for word in item):
        qType = item
        
st = StanfordNERTagger('stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz'
                       ,'stanford-ner/stanford-ner.jar',encoding='utf-8')

tokenized_q = word_tokenize(question)
classified_q = st.tag(tokenized_q)

qTokens = nltk.word_tokenize(question)
qTags = nltk.pos_tag(qTokens)

In [1135]:
kWords = []

for item in classified_q:
    if item[1] != 'O':
        kWords.append(item[0])

for item in qTags:
    if 'NN' in item[1] and item[0] not in kWords:
        kWords.append(item[0])
    if 'JJ' in item[1] and item[0] != 'Which' and item[0] not in kWords:
        kWords.append(item[0])
    if 'bankrupt' in item[0] and item[0] not in kWords:
        kWords.append(item[0])
    if 'CD' in item[1] and item[0] not in kWords:
        kWords.append(item[0])

In [1136]:
kWords

['percentage', 'drop', 'GDP', 'fiscal', 'cliff']

#### Document Selection

In [1137]:
# tf-idf function

def tf(term,document):
    counts = Counter(articles[0].split()).most_common()
    stopword = set(stopwords.words('english'))

    for word in counts: 
        if word[0] not in stopword:
            count = word[1]
            break
            
    value = .5+(.5*document.count(term)/count)
    return value

def idf(term,corpus):
    value = np.log(len(corpus)/len(hm[term]))
    return value

def tf2(term,sentence):
    counts = Counter(", ".join(sentence).split()).most_common()
    stopword = set(stopwords.words('english'))

    for word in counts: 
        if word[0] not in stopword:
            counter = word[1]
            break
            
    value = .5+(.5*sentence.count(term)/counter)
    return value

def idf2(term,corpus):
    count=0
    for sentence in corpus:
        if term in sentence:
            count+=1
    value = np.log(len(corpus)/count)
    return value

In [1138]:
hm = {}

for item in kWords:
    hm[item] = []
    
for item in hm:
    for article in articles:
        if item in article:
            hm[item].append(articles.index(article))   

In [1139]:
scores = []
for word in hm.keys():
    for value in hm[word]:
        scores.append([word,value,tf(word,articles[value])*idf(word,articles)])

In [1140]:
topScore = {}

for word in hm.keys():
    topScore[word] = []

for word in hm.keys():
    for score in scores:
        if score[0] == word:
            topScore[score[0]].append((score[1],score[2]))

In [1141]:
topScoreList = []

for word in topScore.keys():
    topScoreList.append([word,max(topScore[word],key=lambda item:item[1])])

#### Answer Analysis

In [1142]:
hmSen = {}

for item in topScoreList:
    hmSen[item[0]] = []
    hmSen[item[0]].extend(sent_tokenize(articles[item[1][0]]))

In [1143]:
finalSentences = []

for word in hmSen:
    for sentence in hmSen[word]:
        if any(key in sentence for key in kWords):
            finalSentences.append(sentence)

In [1144]:
import re

def generate_ngrams(s, n):
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [1145]:
stopword = set(stopwords.words('english'))
filteredQ = []
filteredQ = [w for w in qTokens if not w in stopword] 
newQ = ' '.join(filteredQ)
bigramQ = generate_ngrams(newQ,2)

In [1146]:
highScore = 0

for sentence in finalSentences:
    filterA = []
    tokenizedA = word_tokenize(sentence)
    filterA = [w for w in tokenizedA if not w in stopword] 
    newA = ' '.join(filterA)
    bigramA = generate_ngrams(newA,2)
    score1 = len(set(bigramQ) & set(bigramA))*2
    score2 = 0
    score3 = 0
    for word in kWords:
        if word in sentence:
            score2+=tf2(word,sentence)*idf2(word,finalSentences)
        else:
            score3+=tf2(word,sentence)*idf2(word,finalSentences)
    finalScore = score1+score2-score3
    if finalScore > highScore:
        highScore = finalScore
        finalAnswer = sentence
        
tokenized_A = word_tokenize(finalAnswer)
classified_A = st.tag(tokenized_A)

aTokens = nltk.word_tokenize(finalAnswer)
aTags = nltk.pos_tag(aTokens)

In [1147]:
finalA = ''
for tuples in classified_A:
    if qType == ['Who', 'Whom', 'Whose']: 
        if tuples[1] == 'PERSON':
            if finalA == '':
                finalA+=(tuples[0])
            else:
                finalA+=(' '+tuples[0])
    if qType == ['Which']:
        if tuples[1] == 'ORGANIZATION':
            if finalA == '':
                finalA+=(tuples[0])
            else:
                finalA+=(' '+tuples[0]) 
                
if finalA == '':
    for tuples in aTags:
        if qType == ['What']:
            if tuples[1] == 'NN':
                if finalA == '':
                    finalA+=(tuples[0])
                else:
                    finalA+=(' '+tuples[0])     
        if qType == ['How few','How great','How many','How little','How much','What percentage']:
            if tuples[1] == 'CD':
                finalA+=(tuples[0])
                break

print(finalA)
print(finalAnswer)

One
Baker concludes: One item worth noting is the GDP report provides zero evidence that "fiscal cliff" concerns had any impact on growth in the quarter.


In [1114]:
finalSentences

['The 12-month federal deficit is down to $723 billion from a record high of $1.5 trillion during February 2010.Â\xa0  In a new publication, we compare federal government spending on goods and services, as reported in the GDP accounts, to total federal outlays asÂ\xa0reportedÂ\xa0in theÂ\xa0Monthly Treasury Statement of Receipts and Outlays.',
 '"A weaker currency helps market-share gains, raises inflation, raises nominal GDP growth, and given Japanâ€™s high operating leverage, raises EPS growth above what investors expect."',
 'Economists were expecting a smaller drop to 15.0.',
 'The percentage of firms reporting increased activity this month (28 percent) was greater than the percentage reporting decreased activity (19 percent).',
 'The percentage of firms reporting increases in employment (19 percent) exceeded the percentage reporting decreases (16 percent).',
 'The factory operating rate moved down 0.1 percentage point to 75.8 percent, a rate 2.9 percentage points below its long-ru